## Implementation using CVX


In [3]:
import cvxpy
import numpy as np

In [4]:
network = 'data/I210'
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
nb_links = graph.shape[0]

In [5]:
f_cvx = cvxpy.Variable(nb_links)
obj_cvx = np.sum(graph[:,3]*f_cvx + 1/2*graph[:,4]*(f_cvx**2) + 1/3*graph[:,5]*(f_cvx**3) + 1/4*graph[:,6]*(f_cvx**4) + 1/5*graph[:,7]*(f_cvx**5))

### TO DO: Define delta (incidence matrix), the paths, A and u

In [9]:
h_cvx = cvxpy.Variable()
u = []

In [10]:
constr = [
    f_cvx == D @ h_cvx,
    h_cvx >= 0,
    A @ h_cvx == demand[:,2],
    f_cvx <= u
]

NameError: name 'D' is not defined

In [11]:
pb = cvxpy.Problem(cvxpy.Minimize(obj_cvx), constr)

NameError: name 'constr' is not defined

In [12]:
pb.solve()

NameError: name 'pb' is not defined